Explicação do problema e dos dados: 
Os dados para esta competição são fornecidos em dois arquivos: train.csve test.csv. O conjunto de treinamento tem 9557 linhas e 143 colunas, enquanto o conjunto de teste tem 23856 linhas e 142 colunas. Cada linha representa um indivíduo e cada coluna é um recurso, exclusivo para o indivíduo ou para a família do indivíduo . O conjunto de treinamento tem uma coluna adicional Target, que representa o nível de pobreza em uma escala de 1 a 4 e é o rótulo da competição. Um valor de 1 é a pobreza mais extrema.

Este é um problema de aprendizado de máquina de classificação multi-classe supervisionado :

Supervisionado : fornecido com os rótulos dos dados de treinamento
Classificação multiclasse: os rótulos são valores discretos com 4 classes
Objetivo
O objetivo é prever a pobreza ao nível do agregado familiar . Recebemos dados em nível individual com cada indivíduo tendo características únicas, mas também informações sobre sua família. Para criar um conjunto de dados para a tarefa, teremos que realizar algumas agregações dos dados individuais para cada família. Além disso, temos que fazer uma previsão para cada indivíduo no conjunto de teste, mas "SOMENTE os chefes de família são usados ​​na pontuação", o que significa que queremos prever a pobreza em uma base familiar.

Observação importante: embora todos os membros de uma família devam ter o mesmo rótulo nos dados de treinamento, existem erros onde os indivíduos na mesma casa têm rótulos diferentes. Nestes casos, somos orientados a usar a etiqueta do chefe de cada família, que pode ser identificada pelas linhas onde parentesco1 == 1.0. Abordaremos como corrigir isso no caderno (para mais informações, dê uma olhada na discussão principal da competição ).

Os Targetvalores representam os níveis de pobreza da seguinte forma:

1 = Extrema pobreza 
2 = Pobreza moderada
3 = Famílias vulneráveis 
4 = Famílias não vulneráveis

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set a few plotting defaults
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 18
plt.rcParams['patch.edgecolor'] = 'k'

In [ ]:
def plotar(variaveis,eixoX,titulo):
    eixoY = []
    for v in variaveis: 
        eixoY.append(df[v].value_counts()[1])
    
    plt.figure(figsize=(20,5))
    sns.barplot(x = eixoX,y = eixoY).set_title(titulo)
    plt.show()

Célula padrão da criação de um novo notebook - importando o numpy e o pandas e processa o diretório para mostrar quais arquivos possui.

In [ ]:
#Carregar os dados dos datasets
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

A base de treino possui 9557 linhas e 143 colunas e a de teste 23856 linhas e 142 colunas. O treino será realizado com 9557 linhas e a previsão será com as 23856 linhas. As linhas que não possuirem a informação de chefe de família serão desconsideradas pelo Kaggle no resultado final.

In [ ]:
# Chefes de família
heads = df.loc[df['parentesco1'] == 1].copy()



# Variáveis para treinamento
train_campos = df.loc[(df['Target'].notnull()) & (df['parentesco1'] == 1), ['Target', 'idhogar']]

# Quantidade de chefes conforme a calissificação
l_counts = train_campos['Target'].value_counts().sort_index()

l_counts


In [ ]:
Parentes = 'Parentesco'
variaveis = 'parentesco1','parentesco2','parentesco3','parentesco4','parentesco5','parentesco6','parentesco7','parentesco8','parentesco9','parentesco10','parentesco11','parentesco12'
eixoX = ['Chefe de família','Cônjugue','Filho','Divorciado','Genro/Nora','Neto','Pai','Sogro','Irmão','Cunhada','Outro Familiar','Outro Não Familiar']
plotar(variaveis,eixoX,Parentes)

Gráfico para verificação da quantidade de individuos e parentesco

In [ ]:
# Verificação de famílias onde os indivíduos no mesmo domicílio têm um nível de pobreza diferente na base de treino
# Agrupa as famílias para verificar os valores únicos
all_equal = df.groupby('idhogar')['Target'].apply(lambda x: x.nunique() == 1)

# Famílias onde as metas não são todas iguais
not_equal = all_equal[all_equal != True]
print('Encontramos {} Individuos na mesma familia no mesmo domícilio que possuem um nível de probreza diferente e precisamos corrigir.'.format(len(not_equal)))

In [ ]:
# Iteração com cada familia
for household in not_equal.index:
    # Localizar a classificação correta do chefe para cada familia
    true_target = int(df[(df['idhogar'] == household) & (df['parentesco1'] == 1.0)]['Target'])
    
    # Definindo a target correta para todos os membros da família
    df.loc[df['idhogar'] == household, 'Target'] = true_target
    
    
# Agrupando a famíliapara decobrir o número de valores únicos
all_equal = df.groupby('idhogar')['Target'].apply(lambda x: x.nunique() == 1)

# Famílias onde as metas não são todas iguais
not_equal = all_equal[all_equal != True]
print('Encontramos {} familias no mesmo domícilio que possuem um nível de probreza diferente.'.format(len(not_equal)))

In [ ]:
households_leader = df.groupby('idhogar')['parentesco1'].sum()

# verificação de familias sem chefe
households_no_head = df.loc[df['idhogar'].isin(households_leader[households_leader == 0].index), :]

print('There are {} households without a head.'.format(households_no_head['idhogar'].nunique()))





Verificação das familias que não possuiam chefes

In [ ]:
# Verificação das famílias sem chefe e com classificação diferente
households_no_head_equal = households_no_head.groupby('idhogar')['Target'].apply(lambda x: x.nunique() == 1)
print('{} Households with no head have different labels.'.format(sum(households_no_head_equal == False)))

In [ ]:
df_all = df.append(test)

df_all.shape

Unindo os data sets de treino e de teste para verificação das características dos dados. Na sequencia dos tratamentos dos dados , as bases de treino e de teste serão separados novamente. Na união temos 33413 linhas e 143 colunas. Foi criada uma coluna target no data ser de teste.

In [ ]:
df_all.drop('area2', axis = 1, inplace = True)

In [ ]:
# Criando a variável com as caracteristicas das paredes da casa
df_all['walls'] = np.argmax(np.array(df_all[['epared1', 'epared2', 'epared3']]),
                           axis = 1)
df_all = df_all.drop(columns = ['epared1', 'epared2', 'epared3'])


In [ ]:
df_all['walls'].value_counts()

Versão 10 - verificamos que houve melhoria na pontuação após a entrega da versão e da utilização da variáve

In [ ]:
# Roof ordinal variable
#df_all['roof'] = np.argmax(np.array(df_all[['etecho1', 'etecho2', 'etecho3']]),
#                           axis = 1)
#df_all = df_all.drop(columns = ['etecho1', 'etecho2', 'etecho3'])

In [ ]:
#df_all['roof'].value_counts()

Versão 11

In [ ]:
# Floor ordinal variable
#df_all['floor'] = np.argmax(np.array(df_all[['eviv1', 'eviv2', 'eviv3']]),
 #                          axis = 1)
#df_all = df_all.drop(columns = ['eviv1', 'eviv2', 'eviv3'])

In [ ]:
#df_all['floor'].value_counts()

Varsão 12

In [ ]:
# Create new feature
#df_all['walls+roof+floor'] = df_all['walls'] + df_all['roof'] + df_all['floor']

In [ ]:
#df_all['walls+roof+floor'].value_counts()

Varsão 13

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values()

In [ ]:
df_all.select_dtypes('object').head()

Verificando as colunas que são do tipo object (que contém texto), pois o modelo trabalha somente com colunas numericas.

In [ ]:
# Analisando os dados da coluna edjefa
df_all['edjefa'].value_counts()

Verificando os registros da chefe de família (feminino), conforme a quantidade de anos de escolaridade. Nesse resultado verificamos a quantidade de 22075 mulheres sem escolaridade, o dado está como "no" e "yes" que é igual a 1 a quantidade de 214, vamos tratar os dados adiante., vamos tratar os dados adiante.

In [ ]:
# Analisando os dados da coluna edjefe
df_all['edjefe'].value_counts()

Verificando os registros do chefe de família (masculino), conforme a quantidade de anos de escolaridade. Nesse resultado verificamos a quantidade de 12818 homens sem escolaridade, o dado está como "no" e "yes" que é igual a 1 a quantidade de 416, vamos tratar os dados adiante.

In [ ]:
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

Sobrescrevendo os dados que estão como yes e no para 1 e 0 para as variáveis edjefa e edjfe

In [ ]:
# Verificando a sobreposição dos dados para o masculino
df_all['edjefe'].value_counts()

In [ ]:
# Verificando a sobreposição dos dados para o Feminino
df_all['edjefa'].value_counts()

In [ ]:
# COntinuação da verificação das variaveis que são do tipo Object
df_all.select_dtypes('object').head()

Pelo resultado e após sobrescrever como yes e no para 1 e 0 para as variáveis edjefa e edjfe, verificamos que essas variáveis não retornaram como object, logo se tornaram numericas. A coluna dependency restou para ser tratada.

In [ ]:
# Verificando a coluna dependence
df_all['dependency'].value_counts().sort_values()

Mediante o resultado, vemos que temos a seguinte quantidade de dados: yes = 7580 e no = 6036. Vamos tratar esses dados para que se tornem como ponto flutuante, assim como os demais da coluna.

In [ ]:
# Sobrepondo os dados da coluna dependency conforme o mapeamento e tranformando em tipo float
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
df_all['dependency'].value_counts()

In [ ]:
df_all['dependency'].isnull().sum()

In [ ]:
# COntinuação da verificação das variaveis que são do tipo Object
df_all.select_dtypes('object').head()

Podemos verificar no resultado acima que não temos mais nenhuma variável que utilizaremos como object. as variáveis ID e idhogar não serão utilizadas.

In [ ]:
# verificando as informações do data set
df_all.info()

O resultado gerado mostra 143 variáveis (considerado muitas variáveis).

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values()

In [ ]:
df_all['Target'].value_counts()

In [ ]:
# Verificando os valores de aluguel (v2a1) para os chefes de familia (parentesco = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Verificando os dados de  v2a1 - pagamento do valor de aluguel
df_all['v2a1'].value_counts()

In [ ]:
df_all['v2a1'].isnull().sum()

In [ ]:
# Verificando os dados de v18q
df_all['v18q'].value_counts()

In [ ]:
df_all['v18q'].isnull().sum()

In [ ]:
# Verificando os dados de v18q1 
df_all['v18q1'].value_counts()

In [ ]:
df_all['v18q1'].isnull().sum()

In [ ]:
# Prenchendo com -1 os valores nulos de v2a1
df_all['v2a1'].fillna(-1, inplace=True)

In [ ]:
# Prenchendo com 0 os valores nulos de v18q1
df_all['v18q1'].fillna(0, inplace=True)

In [ ]:
# Prenchendo com -1 os valores nulos de SQBmeaned, meaneduc e rez_esc
df_all['SQBmeaned'].fillna(-1, inplace=True)
df_all['meaneduc'].fillna(-1, inplace=True)
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
df_all.isnull().sum().sort_values()

In [ ]:
# Feature Engineering

# Vamos criar novas colunas para valores percapita
df_all['phone-pc'] = df_all['qmobilephone'] / df_all['tamviv']
df_all['tablets-pc'] = df_all['v18q1'] / df_all['tamviv']
df_all['rooms-pc'] = df_all['rooms'] / df_all['tamviv']
df_all['rent-pc'] = df_all['v2a1'] / df_all['tamviv']

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]



In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Instanciando o random forest classifier

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['Target'])


In [ ]:
df_all['Target'].value_counts().sort_values()

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Trabalhando com AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
abc = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, random_state=42)
abc.fit(train[feats], train['Target'])
accuracy_score(test['Target'], abc.predict(test[feats]))

In [ ]:
# Trabalhando com GBM
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42)
gbm.fit(train[feats], train['Target'])
accuracy_score(test['Target'], gbm.predict(test[feats]))

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
#Avaliando a importancia de cada coluna
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(25,30))
    
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

Demonstração das variáveis mais importantes para o modelo

In [ ]:
# Limitando o treinamento ao chefe da familia

# Criando um novo dataframe para treinar
heads = train[train['parentesco1'] == 1]

In [ ]:
# Feature Engineering

# Vamos criar novas colunas para valores percapita
heads['hsize-pc'] = heads['hhsize'] / heads['tamviv']
heads['phone-pc'] = heads['qmobilephone'] / heads['tamviv']
heads['tablets-pc'] = heads['v18q1'] / heads['tamviv']
heads['rooms-pc'] = heads['rooms'] / heads['tamviv']
heads['rent-pc'] = heads['v2a1'] / heads['tamviv']

Versão 15

In [ ]:
# Criando um novo modelo
rf2 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

In [ ]:
# Treinando o modelo
rf2.fit(heads[feats], heads['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf2.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Trabalhando com AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
abc = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, random_state=42)
abc.fit(heads[feats], heads['Target'])
accuracy_score(test['Target'], abc.predict(test[feats]))

In [ ]:
# Trabalhando com GBM
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42)
gbm.fit(heads[feats], heads['Target'])
accuracy_score(test['Target'], gbm.predict(test[feats]))

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
#Avaliando a importancia de cada coluna
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(25,30))
    
pd.Series(rf2.feature_importances_, index=feats).sort_values().plot.barh()

Demonstração das variáveis mais importantes para o modelo 2

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
import itertools

# Imprimir a matriz de confusão no modelo de test
print(classification_report(Id, Target))


cmat = confusion_matrix(Id, Target)


plt.figure(figsize = (10,7))
sns.set(font_scale=1.4) # for label size
sns.heatmap(cmat, annot=True, fmt="d") # font size


print('Verdade Negativo {}'.format(cmat[0,0]))
print('Falso Positivo {}'.format(cmat[0,1]))
print('Falso Negativo {}'.format(cmat[1,0]))
print('Verdadeiro Positivo {}'.format(cmat[1,1]))
print('Acurácia: {}'.format(np.divide(np.sum([cmat[0,0],cmat[1,1]]),np.sum(cmat))))
print('Classificação: {}'.format(np.divide(np.sum([cmat[0,1],cmat[1,0]]),np.sum(cmat))))

error_rate = []
acc = []

for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(Id, Target)

    acc.append(knn.score(Id, Target))

    # Plotando o erro

plt.figure(figsize=(10,4))
plt.plot(range(1,40), acc, color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize=10)
plt.title('Accuracia vs. K-Valores')
plt.xlabel('K-Valores')
plt.ylabel('Accuracia')
plt.show()

In [ ]:
cm = confusion_matrix('Target', test['prediction'])

Este trabaho teve como objetivo submeter ao Kaggle um valor melhor para a competição, mas somente os chefes de família são usados na pontuação, o que significa que queremos prever a pobreza em uma base familiar. O valor como parametro foi 0.43719. Após várias tentativas de melhorar o modelo, conseguimos atingir o melhor valor de 0.43768, atingindo o objetivo de melhoria da pontuação.

As análises foram iniciadas a partir do dicionario de dados para o entendimento das caracteristicas das variaveis. O foco inicial foi verificar as famílias onde os indivíduos no mesmo domicílio tinham um nível de pobreza diferente na base de treino, constamos 85 individuos da mesma familia que estão em situaçã de probreza diferente. Ao identificar ajustamos os dados para normalizar a classificação dos individuos da mesma familia com a mesma situação de pobreza.

Com foco no chefe de familia, foi realizada  a verificação das familias que não possuiam chefes. Encontramos 15 familias sem chefes e ajustamos.





